# Analyzing Trump Tweets

In this notebook, I will attempt to perform some analysis on Donald Trump's tweets. I hope to learn how to access Twitter's API to scrape tweet data, as well as some basic natural language processing.

In [ ]:
import tweepy
from tweepy import OAuthHandler
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
#set up access to Twitter API 
consumer_key = 'SsPIsxuj0sGUbSiSPVVUhqY9b'
consumer_secret = 'vQi4cbKyYAnjYjuMFZ79g9i0MJFrgUofkZ7UfDyS2MFMNMYWsU'
access_token = '868623131799412736-cyBXAkrU6fSWWWaIT97st2DUiyBCQvW'
access_secret = 'Ei2jdovTshu9DVW7lbUsGv8CcbdrFKjC7meUSzBDHDHtb'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

Since the Twitter API only returns tweets of one page at a time, we will have to iterate through the pages to grab the entire timeline. This code is adapted from https://gist.github.com/yanofsky/5436496

In [ ]:
#initialize a list to hold all the tweepy Tweets
alltweets = []

#make initial request for most recent tweets (200 is the maximum allowed count)
new_tweets = api.user_timeline(screen_name = 'realDonaldTrump',count=200,include_rts = True)

#save most recent tweets
alltweets.extend(new_tweets)

#save the id of the oldest tweet less one
oldest = alltweets[-1].id - 1

#keep grabbing tweets until there are no tweets left to grab
while len(new_tweets) > 0:
    print("getting tweets before %s" % (oldest))

    #all subsiquent requests use the max_id param to prevent duplicates
    new_tweets = api.user_timeline(screen_name = 'realDonaldTrump',count=200,max_id=oldest,include_rts = False)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #update the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    print("...%s tweets downloaded so far" % (len(alltweets)))


Great, so now we have a list of all the tweet objects. Each tweet object has a wealth of associated metadata, which we can explore:

In [ ]:
print(dir(alltweets[0]))

At this point, I really want to use NLTK's VADER sentiment analysis package to calculate a sentiment score for each tweet. The algorithm calculates a compound "polarity" score for each tweet - that is, it analyzes words that are commonly associated with positive or negative sentiments, and assigns a weighted average of sorts to the tweet. It might be interesting to plot the sentiment of each tweet over time and see if we can find any interesting results. 

Since for now the only information we need is the tweet text itself, let's create a dictionary of the tweet content with its tweet ID so we can reference other metadata later.

In [ ]:
tweetcontent = {}
for tweet in alltweets: 
    tweetcontent[tweet.id] = tweet.text

Cool, now we have a dictionary of tweet IDs and the relevant tweet content. Let's throw this into VADER and see what we find...

In [ ]:
sid = SentimentIntensityAnalyzer()
for key in tweetcontent:
    print(tweetcontent[key])
    ss = sid.polarity_scores(tweetcontent[key])
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()